In [ ]:
## STANDARD PYTHON LIBS
import os
import re
import sys
import cv2
import PIL
import math
import time
import random
import scipy.misc
import skimage.io
import skimage.transform
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## ADDING TO ROOT
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '../'))
sys.path.append(ROOT_DIR)

## CUSTOM LIBS
from src.model import log
import src.utils as utils
import src.model as modellib
from src.config import Config
import src.visualize as visualize

# GPU LIBS
import keras
import tensorflow as tf

print ('TF : ', tf.__version__, '  Keras : ', keras.__version__)


In [ ]:
class MapillaryConfig(Config):
    NAME = "mapillary"

    GPU_COUNT = 1
    IMAGES_PER_GPU = 8
    
    NUM_CLASSES = 1 + 14  # background + 3 shapes
    # [ '1' '17' '23' '24' '25' '26' '27' '28' '29' '30' '31' '32' '33' '8']
    
    IMAGE_MAX_DIM = 512
    
    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask

    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)  # anchor side in pixels
    RPN_ANCHOR_RATIOS = [0.5, 1, 2]
    
    TRAIN_ROIS_PER_IMAGE = 30
    ROI_POSITIVE_RATIO = 0.9
    
    STEPS_PER_EPOCH = 10
    VALIDATION_STEPS = 2


class MapillaryDataset(utils.Dataset):
    
    def __init__(self, datasets_dir, images_list_txt_file, config):
        
        # Base Code (copied) 
        self._image_ids = []
        self.image_info = []
        # Background is always the first class
        self.class_info = [{"source": "", "id": 0, "name": "BG"}]
        self.source_class_ids = {}
        
        # Add config
        self.config = config
        
        # Add classes
        self.add_classes()
        
        # Extract image files paths
        image_files = []
        with open(images_list_txt_file, 'r') as f:
            for line in f:
                image_files.append(line)
        
        # Add images
        for i in range(len(image_files)):
            self.add_image(source=self.dataset,
                           image_id=i,
                           path=os.path.join(datasets_dir, image_files[i]))
        
        # Prepare
        self.prepare()
        
    
    def add_classes(self):
        self.dataset = 'mapillary'
        # self.add_class(self.dataset, 0, "unlabelled")
        self.add_class(self.dataset, 1, "void-ego vehicle")
        self.add_class(self.dataset, 8, "flat-sidewalk")
        self.add_class(self.dataset, 17, "object-pole")
        self.add_class(self.dataset, 23, "sky-sky")
        self.add_class(self.dataset, 24, "human-person")
        self.add_class(self.dataset, 25, "human-rider")
        self.add_class(self.dataset, 26, "vehicle-car")
        self.add_class(self.dataset, 27, "vehicle-truck")
        self.add_class(self.dataset, 28, "vehicle-bus")
        self.add_class(self.dataset, 29, "vehicle-caravan")
        self.add_class(self.dataset, 30, "vehicle-trailer")
        self.add_class(self.dataset, 31, "vehicle-train")
        self.add_class(self.dataset, 32, "vehicle-motorcycle")
        self.add_class(self.dataset, 33, "vehicle-bicycle")
    
    def load_image(self, image_id):
        image = skimage.io.imread(self.image_info[image_id]['path'])
        return self.resize_image_to_MAX_DIM(image)
    
    def load_mask(self, image_id):
        image_file_path = self.image_info[image_id]['path']
        mask_file_path = image_file_name.split('.jpg')[0] + '.npz'
        
        masks = []
        class_ids_orig = []
        class_ids = []
        masks_sparse = np.load(mask_file_path)
        for class_id in masks_sparse:
            for instance_id in masks_sparse[class_id].item():
                mask_class_instance = np.array(masks_sparse[class_id].item()[instance_id].todense())
                masks.append(self.resize_image_to_MAX_DIM(mask_class_instance))
#                 class_ids_orig.append(class_id)
                class_ids.append(self.map_source_class_id(self.dataset + '.' + class_id))
        
        masks = np.array(masks).transpose(1, 2, 0)
        class_ids = np.array(class_ids).astype(np.uint32)
        return masks, class_ids
    
    def resize_image_to_MAX_DIM(self, image, show=False, verbose=False):
        if verbose:
            print ('-->', img.shape, list(np.unique(img)))
        
        # Image dimensions
        if len(image.shape) == 3:
            (r, c, ch) = image.shape
        elif len(image.shape) == 2:
            (r, c)   = image.shape
        
        # Resize image to have max dimension MAX_DIM
        if r > c:
            new_r = self.config.IMAGE_MAX_DIM
            new_c = round(c / r * self.config.IMAGE_MAX_DIM)
        else:
            new_c = self.config.IMAGE_MAX_DIM
            new_r = round(r / c * self.config.IMAGE_MAX_DIM)
        resized_image = skimage.transform.resize(image, (new_r, new_c), preserve_range=True, mode='reflect').astype('uint8')
        
        if verbose:
            print ('-------->', img_trans.shape, list(np.unique(img_trans)))
        if show:
            f,axarr = plt.subplots(1,2, figsize=(15,15))
            axarr[0].imshow(img)
            axarr[1].imshow(img_trans)
        
        return resized_image

if __name__ == "__main__":
    datasets_dir = '/home/play/GOD_DATASET'
    mapillary_train_images_list_txt_file = '/home/play/GOD_DATASET/open_datasets/mapillary_train.txt'
    mapillary_config = MapillaryConfig()
    trainData = MapillaryDataset(datasets_dir, mapillary_train_images_list_txt_file, mapillary_config)

# INFERENCE

In [ ]:
class InferenceConfig(MapillaryConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
model_infer = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=MODEL_DIR)
model_path = model_infer.find_last()[1]

if model_path is not None:
    print(model_path)
    model_infer.load_weights(model_path, by_name=True)

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
model_infer.detect([test_img], verbose=1)